In [ ]:
import os
import sys
import glob
import cv2
import json
import shutil
import numpy as np
from PIL import Image
import google.generativeai as genai
from contextlib import contextmanager
from pdf2image import convert_from_path
try:
    if not hasattr(np, 'float_'): np.float_ = np.float64
    if not hasattr(np, 'float'): np.float = float
    if not hasattr(np, 'int_'): np.int_ = np.int64
    if not hasattr(np, 'int'): np.int = int
    if not hasattr(np, 'bool'): np.bool = bool
    if not hasattr(np, 'bool8'): np.bool8 = np.bool
except Exception as e:
    print(f"Patch warning: {e}")
# ================= ВАША ФУНКЦИЯ ПРОКСИ (без изменений) =================
@contextmanager
def env_proxy(proxy_str):
    api_key, proxy = proxy_str.split('|')
    old_http, old_https, old_api = os.environ.get('http_proxy'), os.environ.get('https_proxy'), os.environ.get('GOOGLE_API_KEY')
    os.environ['http_proxy'], os.environ['https_proxy'], os.environ['GOOGLE_API_KEY'] = proxy, proxy, api_key
    print(f"Временно установлен прокси: {proxy}")
    try:
        yield
    finally:
        if old_http is None: os.environ.pop('http_proxy', None)
        else: os.environ['http_proxy'] = old_http
        if old_https is None: os.environ.pop('https_proxy', None)
        else: os.environ['https_proxy'] = old_https
        if old_api is None: os.environ.pop('GOOGLE_API_KEY', None)
        else: os.environ['GOOGLE_API_KEY'] = old_api
        print("Прокси и временные ключи очищены.")

# ================= НАСТРОЙКИ ПОЛЬЗОВАТЕЛЯ =================
CONFIG_FILE = 'config.txt'
MODEL_NAME = "gemini-2.5-flash"  # Используйте актуальное имя модели

# --- Пути для PDF ---
# 1. УКАЖИТЕ ПУТЬ К ПАПКЕ bin ВНУТРИ poppler
POPPLER_PATH = r'poppler/Library/bin'  # Пример! Укажите ваш путь

# 2. УКАЖИТЕ ПУТЬ К ВАШЕМУ PDF ФАЙЛУ
INPUT_PDF = r'data/chapter1.pdf'

# --- Остальные пути ---
MODEL_PATH = r'data/comictextdetector.pt'
SAVE_DIR = r'data/results'
LOCAL_OCR_MODEL = 'my_manga_ocr_model'

# Временные папки для обработки PDF
TEMP_IMG_DIR = r'data/temp_pdf_images'
TEMP_RES_DIR = r'data/temp_pdf_results'
# =======================================================

# === ПАТЧ ДЛЯ NUMPY ===
try:
    if not hasattr(np, 'float_'): np.float_ = np.float64
except: pass

# === ИМПОРТЫ ПРОЕКТА ===
sys.path.append(os.getcwd())
try:
    from inference import model2annotations
except ImportError:
    print("Ошибка: нет inference.py")
    sys.exit()
from manga_ocr import MangaOcr

def load_config(filepath):
    """Загружает прокси (IP:PORT:USER:PASS) и API ключ из файла."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            proxy_line, api_key = f.readline().strip(), f.readline().strip()
        if not proxy_line or not api_key: return None
        parts = proxy_line.split(':')
        proxy_url = f"http://{parts[2]}:{parts[3]}@{parts[0]}:{parts[1]}" if len(parts) == 4 else proxy_line
        return f"{api_key}|{proxy_url}"
    except FileNotFoundError:
        print(f"ОШИБКА: Файл конфигурации '{filepath}' не найден!")
        return None

def translate_with_gemini(text_data):
    genai.configure() 
    model = genai.GenerativeModel(MODEL_NAME)
    lines_to_send = [{"id": f"{item['file']}_block_{item['id']}", "text": item['text']} for item in text_data]
    prompt = f"""Ты профессиональный переводчик манги с японского на русский. Переведи список реплик, сохраняя контекст. Вот реплики в JSON:
    {json.dumps(lines_to_send, ensure_ascii=False)}
    ТРЕБОВАНИЯ: Верни ответ ТОЛЬКО в формате JSON (список объектов с полями "id" и "translation"). Не пиши ничего лишнего."""
    print(f"Отправка запроса в Gemini через прокси...")
    try:
        response = model.generate_content(prompt)
        raw_text = response.text.strip().replace("```json", "").replace("```", "")
        translated_list = json.loads(raw_text)
        return {item['id']: item['translation'] for item in translated_list}
    except Exception as e:
        print(f"ОШИБКА GEMINI: {e}")
        return {}

def process_pdf():
    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)
    # --- ШАГ 1: Конвертация PDF в картинки ---
    print("\n--- 1. Конвертация PDF в картинки ---")
    try:
        pages = convert_from_path(INPUT_PDF, poppler_path=POPPLER_PATH)
    except Exception as e:
        print(f"Ошибка Poppler: {e}\nПроверьте, что POPPLER_PATH указывает на папку bin!")
        return
    
    print(f"Найдено страниц: {len(pages)}")
    for i, page in enumerate(pages):
        page.save(os.path.join(TEMP_IMG_DIR, f"page_{i:04d}.png"), 'PNG')

    # --- ШАГ 2: Детекция ---
    print("\n--- 2. Детекция текста ---")
    model2annotations(MODEL_PATH, TEMP_IMG_DIR, TEMP_RES_DIR, save_json=False)

    # --- ШАГ 3: OCR ---
    print("\n--- 3. Распознавание текста (OCR) ---")
    image_files = sorted(glob.glob(os.path.join(TEMP_IMG_DIR, '*.png')))
    batch_data = []
    
    mocr = MangaOcr(pretrained_model_name_or_path=LOCAL_OCR_MODEL) if os.path.exists(LOCAL_OCR_MODEL) else MangaOcr()

    for img_path in image_files:
        filename = os.path.basename(img_path)
        name_no_ext = os.path.splitext(filename)[0]
        txt_path = os.path.join(TEMP_RES_DIR, name_no_ext + '.txt')
        if not os.path.exists(txt_path): continue
        
        img = cv2.imread(img_path)
        h_img, w_img = img.shape[:2]
        
        with open(txt_path, 'r', encoding='utf-8') as f:
            blocks = []
            for line in f.readlines():
                parts = line.strip().split()
                if len(parts) < 5: continue
                try: blocks.append(list(map(float, parts[1:5])))
                except: continue
            blocks.sort(key=lambda b: b[1]) # Сортировка сверху-вниз
        
        for i, (x_c, y_c, w, h) in enumerate(blocks):
            x1, y1 = max(0, int((x_c - w/2)*w_img)), max(0, int((y_c - h/2)*h_img))
            x2, y2 = min(w_img, int((x_c + w/2)*w_img)), min(h_img, int((y_c + h/2)*h_img))
            if x2<=x1 or y2<=y1: continue
            
            crop = Image.fromarray(cv2.cvtColor(img[y1:y2, x1:x2], cv2.COLOR_BGR2RGB))
            text = mocr(crop)
            batch_data.append({'file': filename, 'id': i+1, 'text': text})
            print(f"[{filename}] Распознано: {text}")

    if not batch_data:
        print("Текст для перевода не найден.")
        return

    # --- ШАГ 4: Перевод ---
    print("\n--- 4. Перевод через Gemini ---")
    translation_map = translate_with_gemini(batch_data)

    # --- ШАГ 5: Сохранение ---
    output_filename = os.path.splitext(os.path.basename(INPUT_PDF))[0] + '_translated.txt'
    output_file = os.path.join(SAVE_DIR, output_filename)
    print(f"\n--- 5. Сохранение в {output_file} ---")
    
    with open(output_file, 'w', encoding='utf-8') as f:
        current_file = ""
        for item in batch_data:
            unique_id = f"{item['file']}_block_{item['id']}"
            translated_text = translation_map.get(unique_id, "[Ошибка перевода]")
            
            if item['file'] != current_file:
                page_num = int(item['file'].replace('page_', '').replace('.png', '')) + 1
                f.write(f"\n{'='*20}\nСТРАНИЦА {page_num}\n{'='*20}\n")
                current_file = item['file']
            
            f.write(f"[{item['id']}] JP: {item['text']}\n")
            f.write(f"       RU: {translated_text}\n\n")

# === ОСНОВНОЙ ЗАПУСК ===
if __name__ == "__main__":
    # Подготовка и очистка временных папок
    for temp_dir in [TEMP_IMG_DIR, TEMP_RES_DIR]:
        if os.path.exists(temp_dir): shutil.rmtree(temp_dir)
        os.makedirs(temp_dir)

    proxy_config_string = load_config(CONFIG_FILE)
    
    if proxy_config_string:
        try:
            with env_proxy(proxy_config_string):
                process_pdf()
        finally:
            # Гарантированная очистка временных файлов
            print("\n--- Очистка временных файлов ---")
            for temp_dir in [TEMP_IMG_DIR, TEMP_RES_DIR]:
                if os.path.exists(temp_dir): shutil.rmtree(temp_dir)
    else:
        print("Выход из программы из-за ошибки конфигурации.")